# Maki or How to Teach Manim Time Travel

### Setup

In [1]:
import maki

In [2]:
maki.init(maki.RendererImplementation.opengl)

In [3]:
render_driver = maki.RenderDriver("Maki Showcase Window", 1280, 640, maki.vec4(52.0, 58.0, 64.0, 256.0) * (1/256))

[08:52:30] Maki:	Initializing GLFW.


## Raw Cuboids

In [4]:
cuboid = render_driver.add_cuboid_atom()

In [5]:
render_driver.show_cuboid_atom(cuboid, 1, True)

### Colouring

In [6]:
render_driver.color_cuboid_atom(cuboid, 2, maki.vec4(1.0, 1.0, 0.0, 1.0))

In [7]:
render_driver.color_cuboid_atom(cuboid, 3, maki.vec4(1.0, 0.0, 1.0, 1.0))
render_driver.color_cuboid_atom(cuboid, 4, maki.vec4(0.5, 0.5, 0.5, 1.0))
render_driver.color_cuboid_atom(cuboid, 5, maki.vec4(0.8, 0.0, 0.4, 1.0))

[08:52:30] Maki:	Creating GLFW window 'Maki Showcase Window' (1280, 640).


### Translation

In [8]:
render_driver.translate_cuboid_atom(cuboid, 6, maki.vec3(0.0, 1.0, 0.0))

In [9]:
target_delta = maki.vec3(10.0, 10.0, 5.0)
first_frame = 60       # alpha = 0.0
after_last_frame = 200 # alpha = 1.0

alpha_delta = 1 / (after_last_frame - first_frame)
for frame in range(first_frame, after_last_frame):
    cur_delta = target_delta * alpha_delta
    render_driver.translate_cuboid_atom(cuboid, frame, cur_delta)

## Time Travel

In [10]:
target_delta = maki.vec3(0.0, -5.0, 0.0)
first_frame = 5       # alpha = 0.0
after_last_frame = 50 # alpha = 1.0
alpha_delta = 1 / (after_last_frame - first_frame)
for frame in range(first_frame, after_last_frame):
    cur_delta = target_delta * alpha_delta
    render_driver.translate_cuboid_atom(cuboid, frame, cur_delta)

## Abstraction

In [11]:
from abc import abstractmethod

# this is not clean code, it's just a showcase
class Mobject:
    def __init__(self, render_driver: maki.RenderDriver):
        self.render_driver = render_driver
    
    @abstractmethod
    def show(self, frame: int):
        pass
    @abstractmethod
    def hide(self, frame: int):
        pass
    
    @abstractmethod
    def translate(self, delta: maki.vec3, first_frame: int, after_last_frame: int):
        pass


class Cube(Mobject):
    def __init__(self, render_driver: maki.RenderDriver, first_frame: int):
        super().__init__(render_driver)
        self.cuboid_atom = render_driver.add_cuboid_atom()
        self.show(first_frame)
        
    def show(self, frame: int):
        self.render_driver.show_cuboid_atom(self.cuboid_atom, frame, True)
    def hide(self, frame: int):
        self.render_driver.show_cuboid_atom(self.cuboid_atom, frame, False)
        
    def translate(self, delta: maki.vec3, first_frame: int, after_last_frame: int):
        alpha_delta = 1 / (after_last_frame - first_frame)
        for frame in range(first_frame, after_last_frame):
            cur_delta = delta * alpha_delta
            self.render_driver.translate_cuboid_atom(self.cuboid_atom, frame, cur_delta)

In [12]:
c = Cube(render_driver, 5)

In [13]:
c.translate(maki.vec3(0.0, 0.0, 20.0), 10, 300)

# Some more complicated Mobjects

## Big Block of Cubes

In [14]:
import random


# again, this is very unclean code
def get_random_color():
    return maki.vec4(random.uniform(0.0, 1.0), random.uniform(0.0, 1.0), random.uniform(0.0, 1.0), 1.0)

In [15]:
space = 4
frame = 1.0
for x in range(0, 50 * space, space):
    for y in range(0, 50 * space, space):
        for z in range(0, 50 * space, space):
            cube = render_driver.add_cuboid_atom()
            render_driver.show_cuboid_atom(cube, int(frame), True)
            render_driver.translate_cuboid_atom(cube, int(frame), maki.vec3(x-300, y-200, z))
            render_driver.color_cuboid_atom(cube, int(frame), get_random_color())
            frame += 0.005

[08:52:30] Maki:	Creating OpenGL Renderer.
[08:52:30] Maki:	Initializing GLEW.
[08:52:30] Maki:	Shaders linked.
[08:52:30] Maki:	Shaders linked.


## Text

In [16]:
class TextCubes(Mobject):
    cubes = []
    
    def __init__(self, render_driver: maki.RenderDriver, first_frame: int, after_last_frame: int, text: str):
        super().__init__(render_driver)
        alpha_delta = 1 / (after_last_frame - first_frame)
        font = self.load_font()
        rendered_text = self.render_text(text, *font)
        y = 0.0
        spacing = 3.0
        for line in rendered_text:
            x = 0.0
            for char in line:
                if char == "#":
                    self.cubes.append(self.render_driver.add_cuboid_atom())
                    for frame in range(first_frame, after_last_frame):
                        self.render_driver.translate_cuboid_atom(self.cubes[-1], frame, maki.vec3(x, -y, 0) * alpha_delta)
                    self.render_driver.color_cuboid_atom(self.cubes[-1], random.randint(int(first_frame + (after_last_frame - first_frame) / 2), after_last_frame - 1), get_random_color())
                x += spacing
            y += spacing
        self.show(first_frame)
        
    def show(self, frame: int):
        for cube in self.cubes:
            self.render_driver.show_cuboid_atom(cube, frame, True)
    def hide(self, frame: int):
        for cube in self.cubes:
            self.render_driver.show_cuboid_atom(cube, frame, False)
        
    def translate(self, delta: maki.vec3, first_frame: int, after_last_frame: int):
        alpha_delta = 1 / (after_last_frame - first_frame)
        for frame in range(first_frame, after_last_frame):
            cur_delta = delta * alpha_delta
            for cube in self.cubes:
                self.render_driver.translate_cuboid_atom(cube, frame, cur_delta)
    
    def load_font(self):
        chars = {}
        with open(f"block.txt", "r", encoding="utf-8") as file:
            height = int(file.readline().split(":")[-1])
            spacing = int(file.readline().split(":")[-1])
            while line := file.readline():
                if line[0] == "-":
                    code = int(line[1:])
                    char = []
                    for _ in range(0, height):
                        char.append(list(file.readline().strip("\n"))[::2])
                    chars[code] = char
        return (chars, height, spacing)


    def render_text(self, text, font_chars, height, spacing):
        # rows to be printed
        output_lines = []
        for input_line in text.split("\n"):
            # filling lines with pixels of one char after another
            new_output_lines = [[] for _ in range(height)]
            for input_char in input_line:
                # convert input char into array of pixels
                output_char_rows = font_chars[ord(input_char)]
                for new_output_line, output_char_row in zip(new_output_lines, output_char_rows):
                    new_output_line += output_char_row
            output_lines += new_output_lines

        output_strings = ["".join(line).replace("_", "#").replace("|", "#") for line in output_lines]
        return output_strings

In [17]:
t = TextCubes(render_driver, 1, 200, "Hello World")

[08:52:31] Maki:	CuboidAtom Chrono Sync initiated.
[08:52:31] Maki:	CuboidAtom Chrono Sync initiated.
[08:52:31] Maki:	CuboidAtom Chrono Sync initiated.
[08:52:32] Maki:	CuboidAtom Chrono Sync initiated.
[08:52:32] Maki:	CuboidAtom Chrono Sync initiated.
[08:52:32] Maki:	CuboidAtom Chrono Sync initiated.


## Images and Videos

In [18]:
from PIL import Image
import numpy as np
import itertools
import os

# again, garbage code
class ImageRects(Mobject):
    rects = []
    WIDTH = 96
    HEIGHT = 72
    SPACING = 3
    
    def __init__(self, render_driver: maki.RenderDriver, first_frame: int, after_last_frame: int, path: str, z: float):
        super().__init__(render_driver)
        
        image = Image.open(path)
        image = image.resize((self.WIDTH, self.HEIGHT), Image.ANTIALIAS)
        image_data = np.asarray(image)
        
        for (x, y), frame in zip(itertools.product(range(self.WIDTH), range(self.HEIGHT)), np.linspace(first_frame, after_last_frame, num=self.WIDTH*self.HEIGHT, endpoint=False)):
            # create rect
            self.rects.append(self.render_driver.add_quadrilateral_atom())
            # position rect
            self.render_driver.translate_quadrilateral_atom(self.rects[-1], int(frame), maki.vec3(x * self.SPACING, -y * self.SPACING, z))
            # set color
            self.render_driver.color_quadrilateral_atom(self.rects[-1], int(frame), maki.vec4(float(image_data[y][x][0]/256), float(image_data[y][x][1]/256), float(image_data[y][x][2]/256), 1.0))
            # show rect
            self.render_driver.show_quadrilateral_atom(self.rects[-1], int(frame), True)
            
        print(f"created image for '{path}'")
        
    def show(self, frame: int):
        for cube in self.cubes:
            self.render_driver.show_quadrilateral_atom(cube, frame, True)
    def hide(self, frame: int):
        for rect in self.rects:
            self.render_driver.show_quadrilateral_atom(rect, frame, False)
        
    def translate(self, delta: maki.vec3, first_frame: int, after_last_frame: int):
        alpha_delta = 1 / (after_last_frame - first_frame)
        for frame in range(first_frame, after_last_frame):
            cur_delta = delta * alpha_delta
            for rect in self.rects:
                self.render_driver.translate_quadrilateral_atom(rect, frame, cur_delta)

class VideoRects(Mobject):
    images = []
    SPACING = 3
    
    def __init__(self, render_driver: maki.RenderDriver, first_frame: int, after_last_frame: int, path: str, frames):
        super().__init__(render_driver)
        
        frame_space = np.linspace(first_frame, after_last_frame, num=frames+1, endpoint=False)
        
        for frame in range(1, frames+1):
            self.images.append(ImageRects(self.render_driver, frame_space[frame-1], frame_space[frame], os.path.join(path, f"{frame:02d}.png"), frame * self.SPACING))
    
    def show(self, frame: int):
        for image in self.images:
            image.show(frame)
    def hide(self, frame: int):
        for image in self.images:
            image.hide(frame)
    def translate(self, delta: maki.vec3, first_frame: int, after_last_frame: int):
        for image in self.images:
            image.translate(delta, first_frame, after_last_frame)

In [19]:
t = VideoRects(render_driver, 500, 1000, "examples/rick", 31)

created image for 'examples/rick/01.png'
created image for 'examples/rick/02.png'
created image for 'examples/rick/03.png'
created image for 'examples/rick/04.png'
created image for 'examples/rick/05.png'
created image for 'examples/rick/06.png'
created image for 'examples/rick/07.png'
created image for 'examples/rick/08.png'
created image for 'examples/rick/09.png'
created image for 'examples/rick/10.png'
created image for 'examples/rick/11.png'
created image for 'examples/rick/12.png'
created image for 'examples/rick/13.png'
created image for 'examples/rick/14.png'
created image for 'examples/rick/15.png'
created image for 'examples/rick/16.png'
created image for 'examples/rick/17.png'
created image for 'examples/rick/18.png'
created image for 'examples/rick/19.png'
created image for 'examples/rick/20.png'
created image for 'examples/rick/21.png'
created image for 'examples/rick/22.png'
created image for 'examples/rick/23.png'
created image for 'examples/rick/24.png'
created image fo